In [100]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
from nltk.corpus import stopwords

import RAKE
from summa import keywords
from keybert import KeyBERT

from sklearn.metrics import precision_score, recall_score, f1_score

sw = stopwords.words('russian')
m = MorphAnalyzer()
r = Rake()

In [93]:
kw_model = KeyBERT('clips/mfaq')

### 1. Подготовка корпуса 

Для сравнения я выбрала четыре статьи с cyberleninka, в каждой из которых есть раздел "ключевые слова":
- М. С. Самарина: Символика волка в культуре: от Капитолийской волчицы до волка Франциска Ассизского
- Ганьшина Г.В., Чаус Н.В.: Тамбовский волк. Символ или выдумка?
- Карпенко Н.Т.: Экология волка
- Е.А. Воробьевская, Д.В. Политов: О генетической дифферентации волка в Сибири

In [1]:
txts = []
for i in range(1, 5):
    p = r'волки/v' + str(i) + '.txt'
    with open(p, 'r', encoding='utf-8') as f:
        t = f.read()
        txts.append(t)

In [8]:
with open('keywords.txt', 'r', encoding = 'utf-8') as f:
    k = f.read()

In [10]:
k = k[2:].split('@@')
kw = [[k[i], k[i+1]] for i in range(0, len(k)-1, 2)]

In [23]:
def kw_prep(l):
    
    res = []
    lt = l.split(',')
    for i in lt:
        s = i.split()
        word_lemmas = []
        for w in s: # неясно, что делать со штуками типа "отстрел волков", получится "отстрел волк"
            word_lem = m.parse(w)[0].normal_form
            word_lemmas.append(word_lem)
        kword = ' '.join(word_lemmas)
        res.append(kword)

    return res
    

In [28]:
kw_list = [[kw_prep(i[0]), kw_prep(i[1])] for i in kw]

### 2. Оценка пересечения и создание эталона

In [43]:
gold_kw = []
cr = 1
for i in kw_list:
    au_kw = i[0]
    my_kw = i[1]
    match_kw = set(au_kw).intersection(my_kw)
    pr = len(match_kw)/(len(au_kw)+len(my_kw)-len(match_kw))
    
    g = list(set(au_kw + my_kw))
    gold_kw.append(g)
    
    aup = ', '.join(au_kw)
    myp = ', '.join(my_kw)
    
    print(f'ТЕКСТ {cr}:')
    print(f'Токенов в тексте: {len(txts[cr-1].split())}')
    print(f'Оригинальные ключевые слова: {aup}')
    print(f'Мои ключевые слова: {myp}')
    print(f'Доля пересечения: {pr} ')
    print('\n')
    
    cr += 1
    

tot_len = [len(j.split()) for j in txts]
tl = sum(tot_len)
print(f'Общий объем корпуса: {tl} токенов')
    

ТЕКСТ 1:
Токенов в тексте: 2811
Оригинальные ключевые слова: средневековье, миф, архетип, мистика, ересь, менталитет, культура, религия
Мои ключевые слова: волк, символ, символика, культура, миф, мифология, фольклор, легенда, животное, народ, культ, дьявол, религия, алхимия
Доля пересечения: 0.15789473684210525 


ТЕКСТ 2:
Токенов в тексте: 1855
Оригинальные ключевые слова: народный художественный культура, тамбовский сувенир, скульптура тамбовский волк, легенда о тамбовский волк, возрождение, сохранение и развитие традиционный культура родный край
Мои ключевые слова: народный искусство, творчество, тамбов, волк, тамбовский волк, выражение, легенда, народ
Доля пересечения: 0.0 


ТЕКСТ 3:
Токенов в тексте: 1775
Оригинальные ключевые слова: волк, систематик, история, происхождение, морфология, экология, отношение человек к хищник, охота, истребление, регуляция численность, роль волк в биоценоз
Мои ключевые слова: волк, экология, хищник, истребление, численность, охота, животноводство, р

### 3. Извлечение ключевых слов:

In [56]:
def normalize_text(text):
    
    lemmas = []
    for t in simple_word_tokenize(text):
        ls = m.parse(t)[0].normal_form
        lemmas.append(ls)
        
    return ' '.join(lemmas)

In [57]:
norm_txts = [normalize_text(i) for i in txts]

**Rake**

In [54]:
rake = RAKE.Rake(sw)

In [62]:
rake_all = []
for i in norm_txts:
    rake_list = rake.run(i, maxWords=3, minFrequency=2)
    rake_all.append([i[0] for i in rake_list])

**TextRank**

In [71]:
tr_all = []
for i in norm_txts:
    tr_list = keywords.keywords(i, language='russian', additional_stopwords=sw, scores=False)
    tr_all.append(tr_list.split())


**KeyBERT**

In [79]:
bert_all = []
for i in tqdm(range(len(txts))):
    bert_list = kw_model.extract_keywords(txts[i])
    bert_all.append(bert_list)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:47<00:00, 11.77s/it]


In [84]:
keybert_all = []
for i in bert_all:
    el = [j[0] for j in i]
    keybert_all.append(el)

### 4. Морфосинтакические фильтры

In [92]:
full_kw = [rake_all, tr_all, keybert_all]

In [95]:
pos_cmb = [['NOUN'], ['NOUN', 'NOUN'], ['ADJF', 'NOUN']]

In [96]:
filtred_kw = []
for method in full_kw:
    mtd = []
    for text in method:
        text_kw = []
        for kwrd in text:
            pos_seq = []
            for subword in kwrd.split():
                pos_tag = m.parse(subword)[0].tag.POS
                pos_seq.append(pos_tag)
            if pos_seq in pos_cmb:
                text_kw.append(kwrd)
        mtd.append(text_kw)
    filtred_kw.append(mtd)
                    

### 5. Оценка качества

In [131]:
full_scores = {}
mtds = ['rake', 'text_rank', 'keybert']

In [177]:
def metrics_counter(kw, gold_kw=gold_kw, mtds=mtds):
    eps = 1**-5
    
    scores = {}
    for i in range(len(kw)): #methods level
        method_metrics = []

        for j in range(len(kw[i])): # texts level
            match = len(set(kw[i][j]).intersection(gold_kw[j]))
            prec = match/len(kw[i][j])
            rec = match/len(gold_kw[j])
            f1 = (2*prec*rec)/(prec+rec + eps)
            text_metrics = [prec, rec, f1]
            method_metrics.append(text_metrics)

        scores[mtds[i]] = method_metrics

    return scores

In [181]:
full_scores = metrics_counter(full_kw)
filtred_scores = metrics_counter(filtred_kw)

In [182]:
full_dfs = []
for i in full_scores:
    df = pd.DataFrame(full_scores[i], columns = ['precision', 'recall', 'f1'])
    full_dfs.append(df)

In [183]:
flr_dfs = []
for i in filtred_scores:
    df = pd.DataFrame(filtred_scores[i], columns = ['precision', 'recall', 'f1'])
    flr_dfs.append(df)

**без фильтров**

In [185]:
for i in range(len(mtds)):
    print(mtds[i])
    print(full_dfs[i])
    print('\n')

rake
   precision    recall        f1
0   0.086207  0.263158  0.033625
1   0.090909  0.357143  0.044843
2   0.294118  0.588235  0.183824
3   0.039216  0.235294  0.014480


text_rank
   precision    recall        f1
0   0.055794  0.684211  0.043879
1   0.027027  0.285714  0.011765
2   0.071429  0.647059  0.053790
3   0.053846  0.411765  0.030256


keybert
   precision    recall        f1
0        0.4  0.105263  0.055944
1        0.2  0.071429  0.022472
2        0.0  0.000000  0.000000
3        0.0  0.000000  0.000000




**С фильтрами**

In [186]:
for i in range(len(mtds)):
    print(mtds[i])
    print(flr_dfs[i])
    print('\n')

rake
   precision    recall        f1
0   0.142857  0.263158  0.053476
1   0.172414  0.357143  0.080515
2   0.360000  0.529412  0.201743
3   0.103448  0.176471  0.028526


text_rank
   precision    recall        f1
0   0.109244  0.684211  0.083354
1   0.047619  0.285714  0.020408
2   0.133333  0.588235  0.091116
3   0.096774  0.352941  0.047120


keybert
   precision    recall        f1
0       0.50  0.105263  0.065574
1       0.25  0.071429  0.027027
2       0.00  0.000000  0.000000
3       0.00  0.000000  0.000000




### 6. Проблемы:
- выделяются служебные и неполнозначные слова, обозначения и сокращения типа (г) и специальные символы -> дополнить список стоп-слов
- не выделяются сложные именные группы типа "роль волка в биоценозе" выделяются только rake'ом (это нужно скорее учитывать при выборе метода, чем решать как проблему)
- с моей человеческой точки зрения волк должен быть выделен во всех текстах, однако, keybert, например, не выделяет его вообще (вероятно, он слишком частотный(?)) Это полезная фича, если мы анализируем корпус текстов про волков и не хотим видеть волка в каждом списке, но в каком-то outer scope не очень. Можно попробовать как-то занижать частотность таких слов для работы с бертом.
- выделяются слова с сомнительной степенью важности - "продолжать", "демонстрировать", но их можно отфильтровать, задав какой-нибудь порог для скора

In [148]:
for i in range(len(full_kw)): # вывод нефильтрованных слов для иллюстрации
    print(mtds[i])
    for j in full_kw[i]:
        print(j)
    print('\n')

rake
['парализовать взгляд волк', 'волк-оборотень', 'потусторонний мир', 'образ волк', 'капитолийский волчица', 'адский чудовище', 'скандинавский мифология', 'миф', 'волк', 'франциска', 'образ', 'также', 'мочь', 'волчица', 'волков', 'это', 'ересь', 'ад', 'встреча', 'текст', 'писать', 'лес', 'именно', 'ещё', 'литература', 'человек', 'который', 'культура', 'жестокость', 'время', 'данте', 'победа', 'ужасный', 'дьявол', 'губбио', 'описать', 'i', 'любовь', 'м', '\uf02d м', 'св', 'благородство', 'особенность', 'древнеислый', 'связанный', 'вольфганг', 'призрак', 'кроме', 'н', 'хотя', 'усложняться', 'иванов', 'l', 'assisi', '2', '\uf02d', '«', '4']
['издательский дом тгу', 'тамбовский волк', 'наука рф', 'уна-т', 'дата обращение', 'волк', 'сторона', 'скульптура', 'легенда', 'поэтому', 'ю', 'м-', 'город', 'год', 'тамбов', '« вой »', 'работа', 'tambov', 'ещё', 'г', 'символ', 'выдумка', 'сохранение', 'дерево', 'р', 'державин', 'возможно', 'народ', 'человек', 'прилегать', '130х91х23', 'известняк', 